In [1]:
import torch
from torch import nn
from tqdm import tqdm

In [2]:
X = torch.tensor([[1], [2], [3], [4], [5], [6], [7], [8], [9]], dtype = torch.float32)
Y = torch.tensor([[1],[2],[3],[4],[5], [6], [7], [8], [9]], dtype= torch.float32)

In [3]:
model = nn.Linear(1,1)
loss_fn = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr=0.0001) #adjust the learning rate in the optimizer such that loss is minimum

In [4]:
def data_gen(X,Y):
    for m, n in zip(X, Y):
        yield (m, n)

In [5]:
for i in tqdm(range(10000)):
     for x, y in data_gen(X, Y):
        optim.zero_grad() # Always start optimizer with zero gradients
        outputs = model(x) #get the output from the model
        loss = loss_fn(outputs, y) # calculate the loss
        if i % 5000 == 0:
            print(loss.item())
        loss.backward()
        optim.step()

  0%|                  | 33/10000 [00:00<00:31, 320.42it/s]

1.6936932802200317
4.583030700683594
8.866737365722656
14.508563041687012
21.439422607421875
29.54766845703125
38.67112731933594
48.591739654541016
59.03397750854492


 51%|████████        | 5056/10000 [00:13<00:11, 416.24it/s]

0.0017798967892304063
0.0011734478175640106
0.0006912361714057624
0.00033463904401287436
0.00010548513091634959
5.04622221342288e-06
3.299331001471728e-05
0.00018639898917172104
0.00045879604294896126


100%|███████████████| 10000/10000 [00:27<00:00, 361.35it/s]


In [6]:
y_pred = model(torch.tensor([[10], [11], [12]], dtype= torch.float32)) #Converts input to the model as float32
y_pred = torch.tensor(y_pred, dtype=torch.int64) #Converts output data type of int64
print(y_pred)

tensor([[10],
        [11],
        [12]])


C:\Users\praveenchakra.bh\AppData\Local\Temp\ipykernel_26932\378196866.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred, dtype=torch.int64) #Converts output data type of int64
